In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("climate/file1f43b983122_lat=52.25_lng=5.5_period=20000501-20240917.csv", sep=",", skiprows=11)

# Convert datetime column to datetime format
df['datetime_lst'] = pd.to_datetime(df['datetime_lst'])

# Calculate daily mean and max temperatures
daily_max_temp = df.groupby(df['datetime_lst'].dt.date)['t2m'].max()

# Filter days where mean temp >= 15°C
daily_mean_temp = df.groupby(df['datetime_lst'].dt.date)['t2m'].mean()
valid_days = daily_mean_temp[daily_mean_temp >= 15].index

# Create three filtered datasets based on max temperature ranges
range_20_25 = daily_max_temp[(daily_max_temp >= 20) & (daily_max_temp < 25)].index
range_25_30 = daily_max_temp[(daily_max_temp >= 25) & (daily_max_temp < 30)].index
range_above_30 = daily_max_temp[daily_max_temp >= 30].index

# Intersection with valid days (mean temp >= 15)
range_20_25 = valid_days.intersection(range_20_25)
range_25_30 = valid_days.intersection(range_25_30)
range_above_30 = valid_days.intersection(range_above_30)

# Filter datasets
df_20_25 = df[df['datetime_lst'].dt.date.isin(range_20_25)]
df_25_30 = df[df['datetime_lst'].dt.date.isin(range_25_30)]
df_above_30 = df[df['datetime_lst'].dt.date.isin(range_above_30)]

# Save datasets to CSV files
df_20_25.to_csv("climate/temp_20_25.csv", index=False)
df_25_30.to_csv("climate/temp_25_30.csv", index=False)
df_above_30.to_csv("climate/temp_above_30.csv", index=False)

# Visualization function


In [14]:
# Function to filter dataset and create a copy
def filter_dataset(df, days):
    filtered_df = df[df['datetime_lst'].dt.date.isin(days)].copy()
    return filtered_df

# Function to plot heatmap and save as PNG
def plot_heatmap(df, title, filename, vmin, vmax, height):
    df.loc[:, 'hour'] = df['datetime_lst'].dt.hour
    df.loc[:, 'date'] = df['datetime_lst'].dt.date

    pivot_table = df.pivot_table(values='t2m', index='date', columns='hour', aggfunc='mean')

    plt.figure(figsize=(14, height))
    ax = sns.heatmap(pivot_table, cmap="coolwarm", cbar_kws={'label': 'Temperature (°C)'}, linewidths=0.1, vmin=vmin, vmax=vmax)
    plt.title(title)
    plt.xlabel("Hour")
    plt.ylabel("Day")
    plt.xticks(rotation=45)
    plt.tight_layout()

    # Save the heatmap
    plt.savefig(f"climate/{filename}.png", dpi=300)
    plt.close()
    print(f"Saved {filename}.png")

# Plot and save each dataset
plot_heatmap(df_20_25, "Heatmap for Max Temp between 20-25°C", "heatmap_20_25", 15, 25, 40)
plot_heatmap(df_25_30, "Heatmap for Max Temp between 25-30°C", "heatmap_25_30", 15, 30, 30)
plot_heatmap(df_above_30, "Heatmap for Max Temp above 30°C", "heatmap_above_30", 15, 40, 25)

Saved heatmap_20_25.png
Saved heatmap_25_30.png
Saved heatmap_above_30.png
